In [ ]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
!pip install wandb
import wandb
wandb.login(key="279a68e0fd5d16d5893ca46bdc25076ad1f3be50")
wandb.init(project="DLCS6910",entity="cs22m072")

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0
idx = np.arange(train_images.shape[0])
np.random.shuffle(idx)
train_images=train_images[idx]
train_labels=train_labels[idx]
validation_images=train_images[:6000]
validation_labels=train_labels[:6000]
train_images=train_images[6000:]
train_labels=train_labels[6000:]

4422102/4422102 [==============================] - 0s 0us/step


# **Q1**

In [3]:
labels = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

cur_label=0
imgs=[]
lbls=[]
for i in range(len(train_images)):
  if(cur_label==train_labels[i]):
    imgs.append(train_images[i])
    lbls.append(labels[cur_label])
    cur_label+=1
    if(cur_label==10):break

wandb.log({"Image": [wandb.Image(imgs[i], caption=lbls[i]) for i in range(10)]})

# **Q2 Q3**

In [2]:
class Helper:
  def sigmoid(self,layer,d=False):
      if(d==True):
          return self.sigmoid(layer)*(1-self.sigmoid(layer))
      return 1.0/(1.0+np.exp(-1.0*layer))

  def identity(self,layer,d=False):
      if(d==True):
          return np.ones(layer.shape)
      return layer

  def softmax(self,layer,d=False):
      if(d==True):
          s=self.softmax(layer,False);
          return s*(1-s)
      newlayer=(layer-np.max(layer))
      return np.exp(newlayer)/np.sum(np.exp(newlayer))

  def cross_entropy(self,true_output,output):
      return -1.0*np.sum(true_output * np.log(output+1e-9))
  
  def mean_square_error(self,true_output,output):
      return np.sum((true_output-output)**2)

  def tanh(self,layer,d=False):
      if(d==True):
          return 1-np.tanh(layer)**2
      return np.tanh(layer)

  def relu(self,layer,d=False):
      if(d==True):
        return 1. * (layer > 0)
      return layer * (layer > 0)


In [3]:
class sgd:
  def optimize(self,num_hidden_layers,weights,bias,learning_rate,d_weights,d_bias):
    for i in range(1,num_hidden_layers+2):
      weights[i]=weights[i]-learning_rate*d_weights[i]
      bias[i]=bias[i]-learning_rate*d_bias[i]
    return weights,bias

class momentum:
  def __init__(self,neurons_list):
    self.update_w={}
    self.update_b={}
    for i in range(len(neurons_list)-1):
      self.update_w[i+1]=np.zeros((neurons_list[i+1],neurons_list[i]))
      self.update_b[i+1]=np.zeros((neurons_list[i+1],1))
  def optimize(self,num_hidden_layers,weights,bias,learning_rate,d_weights,d_bias,beta):
    for i in range(1,num_hidden_layers+2):
        self.update_w[i]=beta*self.update_w[i]+d_weights[i]
        self.update_b[i]=beta*self.update_b[i]+d_bias[i]

    for i in range(1,num_hidden_layers+2):
      weights[i]=weights[i]-learning_rate*self.update_w[i]
      bias[i]=bias[i]-learning_rate*self.update_b[i]

    return weights,bias

class nag:
  def __init__(self,neurons_list,weights,bias):
    self.update_w={}
    self.update_b={}
    self.weights_use={}
    self.bias_use={}
    for i in range(len(neurons_list)-1):
      self.update_w[i+1]=np.zeros((neurons_list[i+1],neurons_list[i]))
      self.update_b[i+1]=np.zeros((neurons_list[i+1],1))
    self.bias_use=bias.copy()
    self.weights_use=weights.copy()
  
  def optimize(self,num_hidden_layers,weights,bias,learning_rate,d_weights,d_bias,beta):
    for i in range(1,num_hidden_layers+2):
        self.update_w[i]=beta*self.update_w[i]+d_weights[i]
        self.update_b[i]=beta*self.update_b[i]+d_bias[i]

    for i in range(1,num_hidden_layers+2):
      self.weights_use[i]=self.weights_use[i]-learning_rate*self.update_w[i]
      self.bias_use[i]=self.bias_use[i]-learning_rate*self.update_b[i]

    for i in range(1,num_hidden_layers+2):
      weights[i]=self.weights_use[i]-beta*self.update_w[i]
      bias[i]=self.bias_use[i]-beta*self.update_b[i] 

    return weights,bias,self.weights_use,self.bias_use

class rmsprop:
  def __init__(self,neurons_list):
    self.update_w={}
    self.update_b={}
    self.epsilon=1e-6
    for i in range(len(neurons_list)-1):
      self.update_w[i+1]=np.zeros((neurons_list[i+1],neurons_list[i]))
      self.update_b[i+1]=np.zeros((neurons_list[i+1],1))

  def optimize(self,num_hidden_layers,weights,bias,learning_rate,d_weights,d_bias,beta):
    for i in range(1,num_hidden_layers+2):
        self.update_w[i]=beta*self.update_w[i]+(1-beta)*((d_weights[i])**2)
        self.update_b[i]=beta*self.update_b[i]+(1-beta)*((d_bias[i])**2)

    for i in range(1,num_hidden_layers+2):
      weights[i]=weights[i]-learning_rate*d_weights[i]/(np.sqrt(self.update_w[i])+self.epsilon)
      bias[i]=bias[i]-learning_rate*d_bias[i]/(np.sqrt(self.update_b[i])+self.epsilon)
      
    return weights,bias



class adam:
  def __init__(self,neurons_list):
    self.update_w={}
    self.update_b={}
    self.update_what={}
    self.update_bhat={}
    self.momentum_w={}
    self.momentum_b={}
    self.momentum_what={}
    self.momentum_bhat={}
    self.epsilon=1e-6
    for i in range(len(neurons_list)-1):
      self.update_w[i+1]=np.zeros((neurons_list[i+1],neurons_list[i]))
      self.update_b[i+1]=np.zeros((neurons_list[i+1],1))
      self.momentum_w[i+1]=np.zeros((neurons_list[i+1],neurons_list[i]))
      self.momentum_b[i+1]=np.zeros((neurons_list[i+1],1))
  def optimize(self,num_hidden_layers,weights,bias,learning_rate,d_weights,d_bias,beta1,beta2,t):
    for i in range(1,num_hidden_layers+2):
        self.momentum_w[i]=beta1*self.momentum_w[i]+(1-beta1)*((d_weights[i]))
        self.momentum_what[i]=self.momentum_w[i]/(1-beta1**t)

        self.momentum_b[i]=beta1*self.momentum_b[i]+(1-beta1)*((d_bias[i]))
        self.momentum_bhat[i]=self.momentum_b[i]/(1-beta1**t)

        self.update_w[i]=beta2*self.update_w[i]+(1-beta2)*((d_weights[i])**2)
        self.update_what[i]=self.update_w[i]/(1-beta2**t)

        self.update_b[i]=beta2*self.update_b[i]+(1-beta2)*((d_bias[i])**2)
        self.update_bhat[i]=self.update_b[i]/(1-beta2**t)
  

    for i in range(1,num_hidden_layers+2):
      weights[i]=weights[i]-learning_rate*self.momentum_what[i]/(np.sqrt(self.update_what[i])+self.epsilon)
      bias[i]=bias[i]-learning_rate*self.momentum_bhat[i]/(np.sqrt(self.update_bhat[i])+self.epsilon)
      
    return weights,bias
    
    

class nadam:
  def __init__(self,neurons_list):
    self.update_w={}
    self.update_b={}
    self.update_what={}
    self.update_bhat={}
    self.momentum_w={}
    self.momentum_b={}
    self.momentum_what={}
    self.momentum_bhat={}
    self.epsilon=1e-6
    for i in range(len(neurons_list)-1):
      self.update_w[i+1]=np.zeros((neurons_list[i+1],neurons_list[i]))
      self.update_b[i+1]=np.zeros((neurons_list[i+1],1))
      self.momentum_w[i+1]=np.zeros((neurons_list[i+1],neurons_list[i]))
      self.momentum_b[i+1]=np.zeros((neurons_list[i+1],1))
  def optimize(self,num_hidden_layers,weights,bias,learning_rate,d_weights,d_bias,beta1,beta2,t):
    for i in range(1,num_hidden_layers+2):
        self.momentum_w[i]=beta1*self.momentum_w[i]+(1-beta1)*((d_weights[i]))
        self.momentum_what[i]=self.momentum_w[i]/(1-beta1**t)

        self.momentum_b[i]=beta1*self.momentum_b[i]+(1-beta1)*((d_bias[i]))
        self.momentum_bhat[i]=self.momentum_b[i]/(1-beta1**t)

        self.update_w[i]=beta2*self.update_w[i]+(1-beta2)*((d_weights[i])**2)
        self.update_what[i]=self.update_w[i]/(1-beta2**t)

        self.update_b[i]=beta2*self.update_b[i]+(1-beta2)*((d_bias[i])**2)
        self.update_bhat[i]=self.update_b[i]/(1-beta2**t)
  

    for i in range(1,num_hidden_layers+2):
      weights[i]=weights[i]-learning_rate/(np.sqrt(self.update_what[i])+self.epsilon)*(beta1*self.momentum_what[i]+(1-beta1)*d_weights[i]/(1-beta1**t))
      bias[i]=bias[i]-learning_rate/(np.sqrt(self.update_bhat[i])+self.epsilon)*(beta1*self.momentum_bhat[i]+(1-beta1)*d_bias[i]/(1-beta1**t))
      
    return weights,bias
  






In [4]:
class NeuralNetwork:
  actobj=Helper()
  def __init__(self,num_hidden_layers=3,num_neurons_in_hidden_layer=64,learning_rate=0.001,epoch=10,batch=64,activation="tanh",beta=0.9,beta1=0.999,optimizer="rmsprop",init_strat="xavier",weight_decay=0,loss_type="cross_entropy"):
    self.num_hidden_layers=num_hidden_layers
    self.num_neurons_in_hidden_layer=num_neurons_in_hidden_layer
    self.learning_rate=learning_rate
    self.epoch=epoch
    self.batch=batch
    self.pre_activation_layer={}
    self.activation_layer={}
    self.weights={}
    self.bias={}
    self.d_pre_activation_layer={}
    self.d_activation_layer={}
    self.d_weights={}
    self.d_bias={}
    self.beta=beta
    self.beta1=beta1
    self.optimizer=optimizer
    self.init_strat=init_strat
    self.weight_decay=weight_decay
    self.loss_type=loss_type
    self.activation_fn=activation
    if(activation=="sigmoid"):self.activation_fn=NeuralNetwork.actobj.sigmoid
    if(activation=="tanh"):self.activation_fn=NeuralNetwork.actobj.tanh
    if(activation=="relu"):self.activation_fn=NeuralNetwork.actobj.relu
    if(activation=="identity"):self.activation_fn=NeuralNetwork.actobj.identity
    if(loss_type=="cross_entropy"): self.loss_fn=NeuralNetwork.actobj.cross_entropy
    if(loss_type=="mean_square_error"): self.loss_fn=NeuralNetwork.actobj.mean_square_error

  def weight_initialization(self):
    self.neurons_list=[self.input_size]

    for i in range(self.num_hidden_layers):
      self.neurons_list.append(self.num_neurons_in_hidden_layer)
    self.neurons_list.append(self.output_size)

    for i in range(len(self.neurons_list)-1):
      if(self.init_strat=="random"):
        self.weights[i+1]=np.random.default_rng().uniform(-1,1,(self.neurons_list[i+1],self.neurons_list[i]))
        self.bias[i+1]=np.random.default_rng().uniform(-1,1,(self.neurons_list[i+1],1))
      elif(self.init_strat=="xavier"):
        self.weights[i+1]=np.random.randn(self.neurons_list[i+1],self.neurons_list[i])*np.sqrt(2/(self.neurons_list[i+1]+self.neurons_list[i]))
        self.bias[i+1]=np.random.randn(self.neurons_list[i+1],1)*np.sqrt(2/(self.neurons_list[i+1]+1))
  
  def forward_propogation(self,x):
    self.activation_layer[0]=x
    for i in range(1,self.num_hidden_layers+1):
      self.pre_activation_layer[i]=self.bias[i]+np.matmul(self.weights[i],self.activation_layer[i-1])
      self.activation_layer[i]=self.activation_fn(self.pre_activation_layer[i])
    self.pre_activation_layer[self.num_hidden_layers+1]=self.bias[self.num_hidden_layers+1]+np.matmul(self.weights[self.num_hidden_layers+1],self.activation_layer[self.num_hidden_layers])
    self.activation_layer[self.num_hidden_layers+1]=NeuralNetwork.actobj.softmax(self.pre_activation_layer[self.num_hidden_layers+1])    
    return self.activation_layer[self.num_hidden_layers+1]

  def back_propogation(self,true_output,output):
    d_pre_activation_layer={}
    d_weights={}
    d_activation_layer={}
    d_bias={}
    if(self.loss_type=="cross_entropy"):
      d_pre_activation_layer[self.num_hidden_layers+1]=-1*(true_output-output)
    elif(self.loss_type=="mean_square_error"):
      d_pre_activation_layer[self.num_hidden_layers+1]=-2*(true_output-output)*(output*(1-output))
    for i in range(self.num_hidden_layers+1,0,-1):
      d_weights[i]=np.outer(d_pre_activation_layer[i],self.activation_layer[i-1].T)
      d_bias[i]=d_pre_activation_layer[i]
      if(i==1):break
      d_activation_layer[i-1]=np.matmul(self.weights[i].T,d_pre_activation_layer[i])
      d_pre_activation_layer[i-1]=np.multiply(d_activation_layer[i-1],self.activation_fn(self.pre_activation_layer[i-1],True))#element multiplication

    return d_weights,d_bias,self.loss_fn(true_output,output)
    

  def flush_gradients(self):
    for i in range(len(self.neurons_list)-1):
      self.d_weights[i+1]=np.zeros((self.neurons_list[i+1],self.neurons_list[i]))
      self.d_bias[i+1]=np.zeros((self.neurons_list[i+1],1))

  def accuracy(self,images,labels):
    count=0
    loss=0
    for i in range(images.shape[0]):
      output=self.forward_propogation(images[i].reshape((-1,1)))
      true_output=np.zeros((self.output_size,1))
      true_output[labels[i]]=1
      if(labels[i]==np.argmax(output)):
        count+=1
      loss+=self.loss_fn(true_output,output)
    
    for i in range(1,self.num_hidden_layers+2):
      loss+=(self.weight_decay/2)*np.sum(np.square(self.weights[i]))

    return count*1.0/images.shape[0], loss/images.shape[0]
    

  def run(self,train_images,train_labels,validation_images,validation_labels):
    self.input_size=train_images.shape[1]*train_images.shape[2]
    self.output_size=10
    self.weight_initialization()

    if self.optimizer=="sgd":
      opt=sgd()
    elif self.optimizer=="momentum":
      opt=momentum(self.neurons_list)
    elif self.optimizer=="rmsprop":
      opt=rmsprop(self.neurons_list)
    elif self.optimizer=="nag":
      opt=nag(self.neurons_list,self.weights,self.bias)
    elif self.optimizer=="adam":
      opt=adam(self.neurons_list)
    elif self.optimizer=="nadam":
      opt=nadam(self.neurons_list)

    t=0
    for epoch_no in range(self.epoch):
      val_loss,tra_loss,tra_acc,val_acc=0,0,0,0
      loss=0
      batch_count=0
      self.flush_gradients()
      for i in range(train_images.shape[0]):
        batch_count+=1
        true_output=np.zeros((self.output_size,1))
        true_output[train_labels[i]]=1
        output=self.forward_propogation(train_images[i].reshape((-1,1)))
        d_weights,d_bias,loss=self.back_propogation(true_output,output)
        for j in range(1,self.num_hidden_layers+2):
            self.d_weights[j]=self.d_weights[j]+d_weights[j]
            self.d_bias[j]=self.d_bias[j]+d_bias[j]


        if(batch_count==self.batch):
          for j in range(1,self.num_hidden_layers+2):
            self.d_weights[j]=self.d_weights[j]+self.weight_decay*self.weights[j]

          self.d_weights = {k: v / self.batch for k, v in self.d_weights.items()}
          self.d_bias = {k: v / self.batch for k, v in self.d_bias.items()}
          batch_count=0
          if self.optimizer=="sgd":
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias)
          elif self.optimizer=="momentum":
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta)
          elif self.optimizer=="rmsprop":
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta)
          elif self.optimizer=="adam":
            t+=1
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta,self.beta1,t)
          elif self.optimizer=="nadam":
            t+=1
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta,self.beta1,t)
          elif self.optimizer=="nag":
            self.weights,self.bias,self.weights_use,self.bias_use=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta)

          self.flush_gradients()

      if(batch_count>0):#remaining
          for j in range(1,self.num_hidden_layers+2):
            self.d_weights[j]=self.d_weights[j]+self.weight_decay*self.weights[j]

          self.d_weights = {k: v / batch_count for k, v in self.d_weights.items()}
          self.d_bias = {k: v / batch_count for k, v in self.d_bias.items()}
          batch_count=0
          if self.optimizer=="sgd":
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias)
          elif self.optimizer=="momentum":
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta)
          elif self.optimizer=="rmsprop":
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta)
          elif self.optimizer=="adam":
            t+=1
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta,self.beta1,t)
          elif self.optimizer=="nadam":
            t+=1
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta,self.beta1,t)
          elif self.optimizer=="nag":
            self.weights,self.bias,self.weights_use,self.bias_use=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta)

          self.flush_gradients()     

      val_acc,val_loss=self.accuracy(validation_images,validation_labels)
      tra_acc,tra_loss=self.accuracy(train_images,train_labels) 
      print(epoch_no+1,'Training loss',tra_loss,'Val loss',val_loss,'Training Accuracy',tra_acc,'Val Accuracy',val_acc)
      wandb.log({"Training loss":tra_loss,'Val loss':val_loss,'Training Accuracy':tra_acc,'Val Accuracy':val_acc})
    if self.optimizer=="nag":
      self.weights=self.weights_use #for nag
      self.bias=self.bias_use #for nag


# **Q7**

In [ ]:
nn=NeuralNetwork(3,64,0.001,10,64,"tanh",0.9,"rmsprop","xavier",0,"cross_entopy")
nn.run(train_images,train_labels,validation_images,validation_labels)
pred=[]
labels = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
for i in range(len(test_images)):
  pred.append(np.argmax(nn.forward_propogation(test_images[i].reshape((-1,1)))))

wandb.log({"confusion_matrix" : wandb.plot.confusion_matrix(probs=None,y_true=test_labels, preds=pred,class_names=labels)})

# **Q10**

In [ ]:
mnist = keras.datasets.mnist
(train_images_m, train_labels_m), (test_images_m, test_labels_m) = mnist.load_data()
train_images_m = train_images_m / 255.0
test_images_m = test_images_m / 255.0
idx = np.arange(train_images_m.shape[0])
np.random.shuffle(idx)
train_images_m=train_images_m[idx]
train_labels_m=train_labels_m[idx]
validation_images_m=train_images_m[:6000]
validation_labels_m=train_labels_m[:6000]
train_images_m=train_images_m[6000:]
train_labels_m=train_labels_m[6000:]

nn1=NeuralNetwork(3,64,0.001,10,64,"tanh",0.9,0.999,"rmsprop","xavier",0,"cross_entropy")
nn1.run(train_images_m,train_labels_m,validation_images_m,validation_labels_m)

nn2=NeuralNetwork(3,128,0.001,10,32,"tanh",0.9,"rmsprop","xavier",0,"cross_entropy")
nn2.run(train_images_m,train_labels_m,validation_images_m,validation_labels_m)

nn3=NeuralNetwork(5,128,0.001,5,32,"tanh",0.9,"rmsprop","xavier",0.0005,"cross_entropy")
nn3.run(train_images_m,train_labels_m,validation_images_m,validation_labels_m)